In [ ]:
# notebooks/data_warehouse_dashboard.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# NBA Data Warehouse Dashboard\n",
    "\n",
    "This notebook provides visualizations and analysis of the NBA data warehouse."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "import seaborn as sns\n",
    "import sys\n",
    "import os\n",
    "from pathlib import Path\n",
    "\n",
    "# Add the project directory to the path\n",
    "project_root = Path().resolve().parent\n",
    "sys.path.append(str(project_root))\n",
    "\n",
    "from src.analyze.warehouse_analysis import NBAWarehouseAnalyzer\n",
    "\n",
    "# Configure plotting\n",
    "plt.style.use('fivethirtyeight')\n",
    "plt.rcParams['figure.figsize'] = (14, 8)\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Initialize the analyzer\n",
    "analyzer = NBAWarehouseAnalyzer()\n",
    "\n",
    "# Set the current season\n",
    "current_season = '2023-24'"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Current Season Standings"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Get standings\n",
    "standings = analyzer.season_standings(current_season)\n",
    "\n",
    "# Display conference standings\n",
    "for conference in standings['conference'].unique():\n",
    "    print(f\"\\n{conference} Conference Standings\\n{'='*30}\")\n",
    "    conf_data = standings[standings['conference'] == conference].reset_index(drop=True)\n",
    "    conf_data.index += 1  # Start counting from 1\n",
    "    display(conf_data[['team_name', 'wins', 'losses', 'win_pct', 'ppg', 'papg', 'point_diff']])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Top Scorers"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Get top scorers\n",
    "top_scorers = analyzer.top_scorers(season=current_season, limit=15)\n",
    "\n",
    "# Display top scorers\n",
    "display(top_scorers)\n",
    "\n",
    "# Create a bar chart of top scorers\n",
    "plt.figure(figsize=(12, 8))\n",
    "bars = plt.barh(top_scorers['player_name'], top_scorers['ppg'], color='navy')\n",
    "plt.xlabel('Points Per Game')\n",
    "plt.title(f'Top NBA Scorers - {current_season} Season')\n",
    "plt.gca().invert_yaxis()  # Invert to show highest scorer at the top\n",
    "plt.grid(axis='x', linestyle='--', alpha=0.7)\n",
    "\n",
    "# Add team names to the bars\n",
    "for i, bar in enumerate(bars):\n",
    "    plt.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2, \n",
    "             top_scorers['team_name'].iloc[i], \n",
    "             va='center', alpha=0.8, fontsize=10)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Team Performance Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Get team performance data\n",
    "team_stats = analyzer.team_performance(season=current_season)\n",
    "\n",
    "# Create a scatter plot of offense vs defense\n",
    "plt.figure(figsize=(12, 10))\n",
    "plt.scatter(team_stats['ppg'], team_stats['ppg_allowed'], s=100, alpha=0.7)\n",
    "\n",
    "# Add team labels\n",
    "for i, txt in enumerate(team_stats['team_name']):\n",
    "    plt.annotate(txt, \n",
    "                 (team_stats['ppg'].iloc[i], team_stats['ppg_allowed'].iloc[i]),\n",
    "                 xytext=(5, 5), textcoords='offset points')\n",
    "\n",
    "# Add quadrant lines (league average)\n",
    "avg_pts = team_stats['ppg'].mean()\n",
    "avg_pts_allowed = team_stats['ppg_allowed'].mean()\n",
    "plt.axvline(x=avg_pts, color='gray', linestyle='--', alpha=0.5)\n",
    "plt.axhline(y=avg_pts_allowed, color='gray', linestyle='--', alpha=0.5)\n",
    "\n",
    "# Add quadrant labels\n",
    "plt.text(team_stats['ppg'].max() - 5, team_stats['ppg_allowed'].min() + 2, 'Good Offense\\nGood Defense', \n",
    "         horizontalalignment='right', verticalalignment='bottom')\n",
    "plt.text(team_stats['ppg'].min() + 2, team_stats['ppg_allowed'].min() + 2, 'Poor Offense\\nGood Defense', \n",
    "         horizontalalignment='left', verticalalignment='bottom')\n",
    "plt.text(team_stats['ppg'].max() - 5, team_stats['ppg_allowed'].max() - 2, 'Good Offense\\nPoor Defense', \n",
    "         horizontalalignment='right', verticalalignment='top')\n",
    "plt.text(team_stats['ppg'].min() + 2, team_stats['ppg_allowed'].max() - 2, 'Poor Offense\\nPoor Defense', \n",
    "         horizontalalignment='left', verticalalignment='top')\n",
    "\n",
    "plt.title(f'NBA Team Offensive vs Defensive Performance - {current_season} Season')\n",
    "plt.xlabel('Points Per Game (Offense)')\n",
    "plt.ylabel('Points Allowed Per Game (Defense)')\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Invert y-axis so \"better\" defense is towards the top\n",
    "plt.gca().invert_yaxis()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Player Performance Trends"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Set the player ID to analyze (example: LeBron James)\n",
    "player_id = \"2544\"  # LeBron James\n",
    "player_trends = analyzer.player_trends(player_id, last_n_games=20)\n",
    "\n",
    "player_name = player_trends['player_name'].iloc[0]\n",
    "print(f\"{player_name}'s Last {len(player_trends)} Games\\n{'='*40}\")\n",
    "display(player_trends)\n",
    "\n",
    "# Create a multi-line plot of key stats\n",
    "plt.figure(figsize=(14, 10))\n",
    "\n",
    "# Plot points, rebounds, assists\n",
    "plt.plot(range(len(player_trends)), player_trends['points'], marker='o', linewidth=2, label='Points')\n",
    "plt.plot(range(len(player_trends)), player_trends['rebounds'], marker='s', linewidth=2, label='Rebounds')\n",
    "plt.plot(range(len(player_trends)), player_trends['assists'], marker='^', linewidth=2, label='Assists')\n",
    "\n",
    "# Add game dates to x-axis\n",
    "plt.xticks(range(len(player_trends)), player_trends['game_date'], rotation=45)\n",
    "\n",
    "plt.title(f\"{player_name}'s Performance Trends - Last {len(player_trends)} Games\")\n",
    "plt.xlabel('Game Date')\n",
    "plt.ylabel('Stats')\n",
    "plt.grid(True, linestyle='--', alpha=0.7)\n",
    "plt.legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Compare Top Players"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Compare multiple players (example: LeBron, Giannis, Embiid, Doncic)\n",
    "player_ids = [\"2544\", \"203507\", \"203954\", \"1629029\"]  # Example player IDs\n",
    "comparison = analyzer.player_comparison(player_ids, season=current_season)\n",
    "\n",
    "display(comparison)\n",
    "\n",
    "# Create a radar chart to compare players\n",
    "categories = ['ppg', 'rpg', 'apg', 'spg', 'bpg']\n",
    "labels = ['Points', 'Rebounds', 'Assists', 'Steals', 'Blocks']\n",
    "\n",
    "# Setup radar chart\n",
    "angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()\n",
    "angles += angles[:1]  # Close the loop\n",
    "\n",
    "fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))\n",
    "\n",
    "for i, player in comparison.iterrows():\n",
    "    # Get stats and close the loop for plotting\n",
    "    values = player[categories].tolist()\n",
    "    values += values[:1]\n",
    "    \n",
    "    # Plot player's stats\n",
    "    ax.plot(angles, values, linewidth=2, label=player['player_name'])\n",
    "    ax.fill(angles, values, alpha=0.1)\n",
    "\n",
    "# Set category labels\n",
    "ax.set_xticks(angles[:-1])\n",
    "ax.set_xticklabels(labels)\n",
    "\n",
    "plt.title('Player Comparison - Key Statistics')\n",
    "plt.legend(loc='upper right')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Close Connection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Close the database connection\n",
    "analyzer.close()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}